En este archivo .ipynb voy a trabajar en la creación y desarrollo del conjunto de datos necesario para crear la pestaña en el dashboard correspondiente a los KPI.
Tengo 2 KPI, y en este archivo me voy a centrar en la creación del segundo, la consigna del mismo es la siguiente:
- Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.
Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100

In [1]:
# Cargamos las librerías y los datos necesarios para trabajar:
import pandas as pd
hechos = pd.read_excel("../Datasets/homicidios.xlsx")
hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


Veamos ahora las columnas que nos sirven para nuestro KPI

In [2]:
hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

Nos sirve:

- 'N_VICTIMAS': Para hacer la sumatoria de víctimas en determinado año
- 'AAAA': Para saber en que año se dieron los siniestros víales
- 'VICTIMA': Para determinar si la víctima iba en moto

In [3]:
# Dejamos únicamente las columnas que vamos a utilizar:
hechos = hechos[["N_VICTIMAS","AAAA","VICTIMA"]]
hechos

,N_VICTIMAS,AAAA,VICTIMA
0,1,2016,MOTO
1,1,2016,AUTO
2,1,2016,MOTO
3,1,2016,MOTO
4,1,2016,MOTO
...,...,...,...
691,1,2021,MOTO
692,1,2021,MOTO
693,1,2021,MOTO
694,1,2021,AUTO


In [5]:
# Antes del filtrado, veamos primero que tipo de datos hay en la columna "VICTIMA"
hechos["VICTIMA"].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

In [6]:
# Vamos a filtrar el Dataframe de hechos ahora por "Victima" == Moto
hechos[hechos["VICTIMA"] == "MOTO"]

,N_VICTIMAS,AAAA,VICTIMA
0,1,2016,MOTO
2,1,2016,MOTO
3,1,2016,MOTO
4,1,2016,MOTO
5,1,2016,MOTO
...,...,...,...
687,1,2021,MOTO
689,1,2021,MOTO
691,1,2021,MOTO
692,1,2021,MOTO


In [9]:
# Ahora voy a hacer la sumatoria de las víctimas por año y estos datos los voy a almacenar en una lista
victimas_en_moto_anuales = []
for año in hechos["AAAA"].unique():
    cantidad_victimas = sum(hechos[hechos["AAAA"] == año]["N_VICTIMAS"])
    victimas_en_moto_anuales.append(cantidad_victimas)

print(victimas_en_moto_anuales)

[146, 140, 149, 104, 81, 97]


In [10]:
# Ahora voy a crear una lista que contenga los años únicos
años = []
for año in hechos["AAAA"].unique():
    años.append(año)

print(años)

[2016, 2017, 2018, 2019, 2020, 2021]


In [11]:
# A partir de estas dos listas, voy a crear un diccionario
diccionario_previo_dataframe = {
    "Año":años,
    "Victimas_moto":victimas_en_moto_anuales
}
diccionario_previo_dataframe

{'Año': [2016, 2017, 2018, 2019, 2020, 2021],
 'Victimas_moto': [146, 140, 149, 104, 81, 97]}

In [12]:
# A partir de este diccionario, voy a crear un dataframe
KPI_2 = pd.DataFrame(diccionario_previo_dataframe)
KPI_2

,Año,Victimas_moto
0,2016,146
1,2017,140
2,2018,149
3,2019,104
4,2020,81
5,2021,97


Finalmente, exporto este dataframe cómo archivo csv para posteriormente usarlo en el dashboard.

In [13]:
KPI_2.to_csv("../Datasets/KPI_2.csv")